In [1]:
using CSV
using Tables
using DataFrames
using Plots

In [ ]:
# import patient info
pat_OS_numMut = DataFrame(CSV.File("pat_OS_numMut.csv"))

In [ ]:
# filter patients with no mutations
pat_OS_numMut = pat_OS_numMut[(pat_OS_numMut.numMut .!= 0), :];

In [8]:
# filesToPlot = String.(pat_OS_numMut[!, 2])
Param_combs=filter(!isdir,readdir("Param_combs_sims"))
indivParams=filter(!isdir,readdir("indivParams"))
# push!(filesToPlot,"WT")

# filesToPlot=unique(filesToPlot)
# filesToPlot

34-element Vector{String}:
 "WT"
 "basal_cRelSynth-linking_1.5"
 "k1_101-apoptosis_0.5"
 "k1_108-apoptosis_0.5"
 "k1_108-apoptosis_1.5"
 "k1_113-apoptosis_1.5"
 "k1_26-cellCycle_0.5"
 "k1_3a-cellCycle_0.5"
 "k1_81-apoptosis_1.5"
 "k1_86-apoptosis_1.5"
 "k1_bcl2Exp-linking_1.5"
 "k1_bcl2Exp-linking_11.390625"
 "k1_bcl2Exp-linking_2.25"
 ⋮
 "k1_s_Blimp1-differentiation_0.5"
 "k1_s_IRF4-linking_0.5"
 "k1_tIkBb-NFkB_1.5"
 "k2_1a-cellCycle_1.5"
 "k2_5-cellCycle_0.5"
 "k2_scalingNumber-NFkB_0.5"
 "k2_scalingNumber-NFkB_1.5"
 "k2_scalingNumber-NFkB_2.25"
 "k2_scalingNumber-NFkB_3.375"
 "k3_7-cellCycle_1.5"
 "kmax_IkBaSynth-NFkB_0.5"
 "kmax_IkBeSynth-NFkB_1.5"

In [ ]:
pathToPlot="Param_comb_sims/"

In [4]:
maxTimeTC=float(60*120)
maxGens=8;

In [ ]:
allData=[]
fileIndex=1
for file in filesToPlot
    if startswith(file,"DLBCL")
        treeArrayFile=pathToPlot*file*"/all_mut/treeStructure.csv"
        graphingArray=CSV.File(treeArrayFile,types=Float64, silencewarnings=true) |> Tables.matrix

    elseif file=="WT"
        treeArrayFile=pathToPlot*file*"/treeStructure.csv"
        graphingArray=CSV.File(treeArrayFile,types=Float64, silencewarnings=true) |> Tables.matrix

    else
        try
            treeArrayFile=pathToPlot*file*"/mut/treeStructure.csv"
            graphingArray=CSV.File(treeArrayFile,types=Float64, silencewarnings=true) |> Tables.matrix

        catch
            treeArrayFile=pathToPlot*"indivParams/"*file*"/mut/treeStructure.csv"
            graphingArray=CSV.File(treeArrayFile,types=Float64, silencewarnings=true) |> Tables.matrix
        end
    end


    cellCountArray=zeros(maxGens,Int64(maxTimeTC))

    for i in 1:size(graphingArray,1)
        thisCellMaxGen=Int64(graphingArray[i,end])
        cellsInThisCellMaxGen=2^(thisCellMaxGen)-1
        lastGenEnd=1
        for thisGen in 1:thisCellMaxGen
            cellsInThisGen=2^(thisGen-1)
            thisGenFateTime=graphingArray[i,thisGen]
            cellCountArray[thisGen,Int64(round(lastGenEnd)):Int64(round(thisGenFateTime))-1]=cellCountArray[thisGen,Int64(round(lastGenEnd)):Int64(round(thisGenFateTime))-1].+cellsInThisGen
            lastGenEnd=graphingArray[i,thisGen]

        end 
    end
    if isempty(allData)
        allData=zeros(length(filesToPlot),size(cellCountArray,2))
    end
    allData[fileIndex,:]=sum(cellCountArray,dims=1)
    fileIndex=fileIndex+1
end


In [19]:
function getData(files, pathToPlot, cond)
    allData=[]
    fileIndex=1
    for file in files
            treeArrayFile=pathToPlot*"/"*file*"/"*cond*"/treeStructure.csv"
            graphingArray=CSV.File(treeArrayFile,types=Float64, silencewarnings=true) |> Tables.matrix

        cellCountArray=zeros(maxGens,Int64(maxTimeTC))

        for i in 1:size(graphingArray,1)
            thisCellMaxGen=Int64(graphingArray[i,end])
            cellsInThisCellMaxGen=2^(thisCellMaxGen)-1
            lastGenEnd=1
            for thisGen in 1:thisCellMaxGen
                cellsInThisGen=2^(thisGen-1)
                thisGenFateTime=graphingArray[i,thisGen]
                cellCountArray[thisGen,Int64(round(lastGenEnd)):Int64(round(thisGenFateTime))-1]=cellCountArray[thisGen,Int64(round(lastGenEnd)):Int64(round(thisGenFateTime))-1].+cellsInThisGen
                lastGenEnd=graphingArray[i,thisGen]

            end 
        end
        if isempty(allData)
            allData=zeros(length(files),size(cellCountArray,2))
        end
        allData[fileIndex,:]=sum(cellCountArray,dims=1)
        fileIndex=fileIndex+1
    end
    df=DataFrame(allData',Symbol.(files))
    df[!, :ID] = 1:7200
    CSV.write(pathToPlot*"_outputFile.csv",df)
    return(df)
end

getData (generic function with 1 method)

In [20]:
param_combs_df = getData(Param_combs, "Param_combs_sims", "mut_comb")

Row,basal_cRelSynth-linking:k1_3a-cellCycle,basal_cRelSynth-linking:k1_e_pax5-differentiation,basal_cRelSynth-linking:k1_mycExp-linking,basal_cRelSynth-linking:k1_mycTsl-linking,basal_cRelSynth-linking:k1_s_Bcl6-differentiation,basal_cRelSynth-linking:k1_s_IRF4-linking,basal_cRelSynth-linking:kmax_IkBaSynth-NFkB,k1_101-apoptosis:basal_cRelSynth-linking,k1_101-apoptosis:k1_108-apoptosis,k1_101-apoptosis:k1_113-apoptosis,k1_101-apoptosis:k1_26-cellCycle,k1_101-apoptosis:k1_3a-cellCycle,k1_101-apoptosis:k1_e_pax5-differentiation,k1_101-apoptosis:k1_mycExp-linking,k1_101-apoptosis:k1_mycTsl-linking,k1_101-apoptosis:k1_s_Bcl6-differentiation,k1_101-apoptosis:k1_s_Blimp1-differentiation,k1_101-apoptosis:k1_s_IRF4-linking,k1_101-apoptosis:k1_tIkBb-NFkB,k1_101-apoptosis:k2_1a-cellCycle,k1_101-apoptosis:k2_5-cellCycle,k1_101-apoptosis:k2_scalingNumber-NFkB,k1_101-apoptosis:k3_7-cellCycle,k1_101-apoptosis:kmax_IkBaSynth-NFkB,k1_101-apoptosis:kmax_IkBeSynth-NFkB,k1_108-apoptosis:basal_cRelSynth-linking,k1_108-apoptosis:k1_26-cellCycle,k1_108-apoptosis:k1_3a-cellCycle,k1_108-apoptosis:k1_e_pax5-differentiation,k1_108-apoptosis:k1_mycExp-linking,k1_108-apoptosis:k1_mycTsl-linking,k1_108-apoptosis:k1_s_Bcl6-differentiation,k1_108-apoptosis:k1_s_Blimp1-differentiation,k1_108-apoptosis:k1_s_IRF4-linking,k1_108-apoptosis:kmax_IkBaSynth-NFkB,k1_113-apoptosis:basal_cRelSynth-linking,k1_113-apoptosis:k1_108-apoptosis,k1_113-apoptosis:k1_26-cellCycle,k1_113-apoptosis:k1_3a-cellCycle,k1_113-apoptosis:k1_e_pax5-differentiation,k1_113-apoptosis:k1_mycExp-linking,k1_113-apoptosis:k1_mycTsl-linking,k1_113-apoptosis:k1_s_Bcl6-differentiation,k1_113-apoptosis:k1_s_Blimp1-differentiation,k1_113-apoptosis:k1_s_IRF4-linking,k1_113-apoptosis:k1_tIkBb-NFkB,k1_113-apoptosis:k2_scalingNumber-NFkB,k1_113-apoptosis:kmax_IkBaSynth-NFkB,k1_113-apoptosis:kmax_IkBeSynth-NFkB,k1_26-cellCycle:basal_cRelSynth-linking,k1_26-cellCycle:k1_3a-cellCycle,k1_26-cellCycle:k1_e_pax5-differentiation,k1_26-cellCycle:k1_mycExp-linking,k1_26-cellCycle:k1_mycTsl-linking,k1_26-cellCycle:k1_s_Bcl6-differentiation,k1_26-cellCycle:k1_s_IRF4-linking,k1_26-cellCycle:kmax_IkBaSynth-NFkB,k1_3a-cellCycle:k1_e_pax5-differentiation,k1_3a-cellCycle:k1_mycTsl-linking,k1_3a-cellCycle:k1_s_IRF4-linking,k1_3a-cellCycle:kmax_IkBaSynth-NFkB,k1_81-apoptosis:k1_101-apoptosis,k1_81-apoptosis:k1_113-apoptosis,k1_81-apoptosis:k1_26-cellCycle,k1_81-apoptosis:k1_3a-cellCycle,k1_81-apoptosis:k1_86-apoptosis,k1_81-apoptosis:k1_bcl2Exp-linking,k1_81-apoptosis:k1_e_pax5-differentiation,k1_81-apoptosis:k1_mycExp-linking,k1_81-apoptosis:k1_mycTsl-linking,k1_81-apoptosis:k1_s_Bcl6-differentiation,k1_81-apoptosis:k1_s_Blimp1-differentiation,k1_81-apoptosis:k1_s_IRF4-linking,k1_81-apoptosis:k1_tIkBb-NFkB,k1_81-apoptosis:k2_1a-cellCycle,k1_81-apoptosis:k2_5-cellCycle,k1_81-apoptosis:k3_7-cellCycle,k1_81-apoptosis:kmax_IkBaSynth-NFkB,k1_81-apoptosis:kmax_IkBeSynth-NFkB,k1_86-apoptosis:basal_cRelSynth-linking,k1_86-apoptosis:k1_101-apoptosis,k1_86-apoptosis:k1_108-apoptosis,k1_86-apoptosis:k1_113-apoptosis,k1_86-apoptosis:k1_26-cellCycle,k1_86-apoptosis:k1_3a-cellCycle,k1_86-apoptosis:k1_bcl2Exp-linking,k1_86-apoptosis:k1_e_pax5-differentiation,k1_86-apoptosis:k1_mycExp-linking,k1_86-apoptosis:k1_mycTsl-linking,k1_86-apoptosis:k1_s_Bcl6-differentiation,k1_86-apoptosis:k1_s_Blimp1-differentiation,k1_86-apoptosis:k1_s_IRF4-linking,k1_86-apoptosis:k1_tIkBb-NFkB,k1_86-apoptosis:k2_1a-cellCycle,k1_86-apoptosis:k2_5-cellCycle,k1_86-apoptosis:k3_7-cellCycle,k1_86-apoptosis:kmax_IkBaSynth-NFkB,k1_86-apoptosis:kmax_IkBeSynth-NFkB,k1_bcl2Exp-linking:basal_cRelSynth-linking,k1_bcl2Exp-linking:k1_101-apoptosis,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Floa

In [21]:
indivParams_df = getData(indivParams, "indivParams", "mut")

Row,WT,basal_cRelSynth-linking_1.5,k1_101-apoptosis_0.5,k1_108-apoptosis_0.5,k1_108-apoptosis_1.5,k1_113-apoptosis_1.5,k1_26-cellCycle_0.5,k1_3a-cellCycle_0.5,k1_81-apoptosis_1.5,k1_86-apoptosis_1.5,k1_bcl2Exp-linking_1.5,k1_bcl2Exp-linking_11.390625,k1_bcl2Exp-linking_2.25,k1_bcl2Exp-linking_3.375,k1_bcl2Exp-linking_5.0625,k1_e_pax5-differentiation_0.25,k1_e_pax5-differentiation_0.5,k1_mycExp-linking_1.5,k1_mycTsl-linking_1.5,k1_s_Bcl6-differentiation_1.5,k1_s_Bcl6-differentiation_2.25,k1_s_Blimp1-differentiation_0.25,k1_s_Blimp1-differentiation_0.5,k1_s_IRF4-linking_0.5,k1_tIkBb-NFkB_1.5,k2_1a-cellCycle_1.5,k2_5-cellCycle_0.5,k2_scalingNumber-NFkB_0.5,k2_scalingNumber-NFkB_1.5,k2_scalingNumber-NFkB_2.25,k2_scalingNumber-NFkB_3.375,k3_7-cellCycle_1.5,kmax_IkBaSynth-NFkB_0.5,kmax_IkBeSynth-NFkB_1.5,ID
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64
1,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,1
2,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,2
3,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,3
4,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,4
5,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,5
6,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,6
7,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,7
8,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,8
9,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,9


In [22]:
combsData = rightjoin(param_combs_df,indivParams_df,on=:ID)

Row,basal_cRelSynth-linking:k1_3a-cellCycle,basal_cRelSynth-linking:k1_e_pax5-differentiation,basal_cRelSynth-linking:k1_mycExp-linking,basal_cRelSynth-linking:k1_mycTsl-linking,basal_cRelSynth-linking:k1_s_Bcl6-differentiation,basal_cRelSynth-linking:k1_s_IRF4-linking,basal_cRelSynth-linking:kmax_IkBaSynth-NFkB,k1_101-apoptosis:basal_cRelSynth-linking,k1_101-apoptosis:k1_108-apoptosis,k1_101-apoptosis:k1_113-apoptosis,k1_101-apoptosis:k1_26-cellCycle,k1_101-apoptosis:k1_3a-cellCycle,k1_101-apoptosis:k1_e_pax5-differentiation,k1_101-apoptosis:k1_mycExp-linking,k1_101-apoptosis:k1_mycTsl-linking,k1_101-apoptosis:k1_s_Bcl6-differentiation,k1_101-apoptosis:k1_s_Blimp1-differentiation,k1_101-apoptosis:k1_s_IRF4-linking,k1_101-apoptosis:k1_tIkBb-NFkB,k1_101-apoptosis:k2_1a-cellCycle,k1_101-apoptosis:k2_5-cellCycle,k1_101-apoptosis:k2_scalingNumber-NFkB,k1_101-apoptosis:k3_7-cellCycle,k1_101-apoptosis:kmax_IkBaSynth-NFkB,k1_101-apoptosis:kmax_IkBeSynth-NFkB,k1_108-apoptosis:basal_cRelSynth-linking,k1_108-apoptosis:k1_26-cellCycle,k1_108-apoptosis:k1_3a-cellCycle,k1_108-apoptosis:k1_e_pax5-differentiation,k1_108-apoptosis:k1_mycExp-linking,k1_108-apoptosis:k1_mycTsl-linking,k1_108-apoptosis:k1_s_Bcl6-differentiation,k1_108-apoptosis:k1_s_Blimp1-differentiation,k1_108-apoptosis:k1_s_IRF4-linking,k1_108-apoptosis:kmax_IkBaSynth-NFkB,k1_113-apoptosis:basal_cRelSynth-linking,k1_113-apoptosis:k1_108-apoptosis,k1_113-apoptosis:k1_26-cellCycle,k1_113-apoptosis:k1_3a-cellCycle,k1_113-apoptosis:k1_e_pax5-differentiation,k1_113-apoptosis:k1_mycExp-linking,k1_113-apoptosis:k1_mycTsl-linking,k1_113-apoptosis:k1_s_Bcl6-differentiation,k1_113-apoptosis:k1_s_Blimp1-differentiation,k1_113-apoptosis:k1_s_IRF4-linking,k1_113-apoptosis:k1_tIkBb-NFkB,k1_113-apoptosis:k2_scalingNumber-NFkB,k1_113-apoptosis:kmax_IkBaSynth-NFkB,k1_113-apoptosis:kmax_IkBeSynth-NFkB,k1_26-cellCycle:basal_cRelSynth-linking,k1_26-cellCycle:k1_3a-cellCycle,k1_26-cellCycle:k1_e_pax5-differentiation,k1_26-cellCycle:k1_mycExp-linking,k1_26-cellCycle:k1_mycTsl-linking,k1_26-cellCycle:k1_s_Bcl6-differentiation,k1_26-cellCycle:k1_s_IRF4-linking,k1_26-cellCycle:kmax_IkBaSynth-NFkB,k1_3a-cellCycle:k1_e_pax5-differentiation,k1_3a-cellCycle:k1_mycTsl-linking,k1_3a-cellCycle:k1_s_IRF4-linking,k1_3a-cellCycle:kmax_IkBaSynth-NFkB,k1_81-apoptosis:k1_101-apoptosis,k1_81-apoptosis:k1_113-apoptosis,k1_81-apoptosis:k1_26-cellCycle,k1_81-apoptosis:k1_3a-cellCycle,k1_81-apoptosis:k1_86-apoptosis,k1_81-apoptosis:k1_bcl2Exp-linking,k1_81-apoptosis:k1_e_pax5-differentiation,k1_81-apoptosis:k1_mycExp-linking,k1_81-apoptosis:k1_mycTsl-linking,k1_81-apoptosis:k1_s_Bcl6-differentiation,k1_81-apoptosis:k1_s_Blimp1-differentiation,k1_81-apoptosis:k1_s_IRF4-linking,k1_81-apoptosis:k1_tIkBb-NFkB,k1_81-apoptosis:k2_1a-cellCycle,k1_81-apoptosis:k2_5-cellCycle,k1_81-apoptosis:k3_7-cellCycle,k1_81-apoptosis:kmax_IkBaSynth-NFkB,k1_81-apoptosis:kmax_IkBeSynth-NFkB,k1_86-apoptosis:basal_cRelSynth-linking,k1_86-apoptosis:k1_101-apoptosis,k1_86-apoptosis:k1_108-apoptosis,k1_86-apoptosis:k1_113-apoptosis,k1_86-apoptosis:k1_26-cellCycle,k1_86-apoptosis:k1_3a-cellCycle,k1_86-apoptosis:k1_bcl2Exp-linking,k1_86-apoptosis:k1_e_pax5-differentiation,k1_86-apoptosis:k1_mycExp-linking,k1_86-apoptosis:k1_mycTsl-linking,k1_86-apoptosis:k1_s_Bcl6-differentiation,k1_86-apoptosis:k1_s_Blimp1-differentiation,k1_86-apoptosis:k1_s_IRF4-linking,k1_86-apoptosis:k1_tIkBb-NFkB,k1_86-apoptosis:k2_1a-cellCycle,k1_86-apoptosis:k2_5-cellCycle,k1_86-apoptosis:k3_7-cellCycle,k1_86-apoptosis:kmax_IkBaSynth-NFkB,k1_86-apoptosis:kmax_IkBeSynth-NFkB,k1_bcl2Exp-linking:basal_cRelSynth-linking,k1_bcl2Exp-linking:k1_101-apoptosis,⋯
,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,

# Now let's plot patient data

In [ ]:
#### Load patient data
patients=unique(filter(n->startswith(n,"DLBCL"),names(df)))

In [ ]:
cA=palette(:seaborn_colorblind,20)
#### Mut/Param data
muts2Params = DataFrame(CSV.File("muts2ParamsNEW.csv"))
# concatenate gene and alteration
muts2Params[!, "GeneAlt"] = string.(muts2Params[!, "GENE"], ":", muts2Params[!, "ALTERATION"])
## remove zeros
muts2Params = muts2Params[(muts2Params.PARAMETER .!= "0"), :];
#### Load combined output data

In [ ]:
CombOp = DataFrame(CSV.File("combinedOutput2_noDupes.csv"))
# remove missing data
CombOp = CombOp[completecases(CombOp), :]
# concatenate gene and alteration
CombOp[!, "GeneAlt"] = string.(CombOp[!, "GENE"], ":", CombOp[!, "ALTERATION"]);

In [ ]:
function GenConds(muts)
    Conds = DataFrame(Param = Any[], Val = Float64[])
        for i in 1:length(muts)        
            # check genes are in dictionary
            if haskey(param_mut, muts[i]) == true
                mut=muts[i]
                val=param_vals[mut]
                param=param_mut[mut]
                ParamVal = DataFrame(Param=param, Val=val)
                append!(Conds, ParamVal)
            else 
                continue
            end
        end
    return(Conds)
end

function GenConds2(muts_count)
    p2C=[]
    mA=[]
    for i in 1:nrow(muts_count)
        param=muts_count[i,1]
        push!(p2C, param)
        val=muts_count[i,2]
        NUM=muts_count[i,3]
        PV=val^NUM
        push!(mA, PV)
    end
    return(p2C, mA)
end        

# create mut dictionary
param_mut=Dict() 
# populate mut dictionary
for i in 1:nrow(muts2Params)
    GeneAlt = muts2Params[i,:GeneAlt]
    Param = muts2Params[i,:PARAMETER]
    param_mut[GeneAlt]=Param
end    

# create vals dictionary
param_vals=Dict()
# populate vals dictionary
for i in 1:nrow(muts2Params)
    GeneAlt = muts2Params[i,:GeneAlt]
    Val = muts2Params[i,:AMOUNT]
    param_vals[GeneAlt]=Val
end   

# create gene:alt dictionary
gene_alt=Dict() 
# populate mut dictionary
for i in 1:nrow(muts2Params)
    GeneAlt = muts2Params[i,:GeneAlt]
    Param = muts2Params[i,:PARAMETER]
    gene_alt[Param]=GeneAlt
end    

In [ ]:
# get data for individual patient
function getPatConds(patient)
    patient_df = subset(CombOp, :PATIENT => ByRow(==(patient)))
    
    # we only need the dataframe to contain those mutations we think do something so filter out the rest.
    filter!(:GeneAlt => n -> haskey(param_mut, n), patient_df)
    
    # get mutations
    muts = Vector(patient_df[!, :GeneAlt])
    conds = GenConds(muts)
    muts_count = combine(groupby(conds, [:Param, :Val]), nrow => :count)
        
    ######################
    ### Set conditions ###
    ######################
    
    # set patient name
    try
        patient = patient_df[1,1]
    catch
        println("patient: "*patient*" has no mutations.")
        return (["WT",patient],["WT","all_muts"])
    end
    
    conditions = ["all_mut"]
    
    # get conditions
    conds_run = GenConds2(muts_count)
    
    condsToPlot = Any["WT"]
    params = conds_run[1]
    values = conds_run[2]
    
    for i in 1:length(conds_run[1])
        folderName = string.(params[i], "_", values[i])
        push!(condsToPlot, folderName)
    end
    push!(condsToPlot, patient)
    
    conds_muts = deepcopy(conds)
    conds_muts[!, "muts"] = muts
    
    GeneAlt = Any["WT"]
    for i in 1:length(params)
        param_df = subset(conds_muts, :Param => ByRow(==(params[i])))
        label=""
        thisIndex=1
        for row in eachrow(param_df)
            if label==""
                label=string.(param_df.muts[1])
            else
                label=label*"+"*string.(param_df.muts[thisIndex])
            end
            thisIndex+=1
        end
        push!(GeneAlt, label)
    end
    push!(GeneAlt, "all_muts")    
    
   return(condsToPlot, GeneAlt)
end

In [ ]:
function plotCellNumbers(thisPatient)
    gr()
    # get conditions and names for conditions
    (conditionsForPat,namesForPatConds)=getPatConds(thisPatient)
    p1=plot()
    conditionIndex=1

    cA=palette(:seaborn_colorblind,length(conditionsForPat))

    for condition in conditionsForPat
        #if this is the all muts condition, use the black line we saved
        if conditionIndex==length(conditionsForPat)
            thisLineColor=:black
        else
            #otherwise plot in whatever colour is next in the color array with green removed
            thisLineColor=cA[conditionIndex]
        end
        if conditionIndex==length(conditionsForPat)
            linewidth=6
        elseif conditionIndex==1
            linewidth=6
        else
            linewidth=3
        end
        # just plot the cell numebrs in this condition
        plot!(df[!,Symbol(condition)],label=namesForPatConds[conditionIndex],linecolor=thisLineColor, linealpha=0.75,linewidth=linewidth)
        
        conditionIndex+=1
    end

    #formating 
    plot!(xticks = (0:60*24:maxTimeTC, string.(0:24:Int64(maxTimeTC))))
    plot!(ylabel="cell number")
    plot!(xlabel="time (h)")
    plot!(legend=:right,dpi=300,margin=10Plots.mm)
    
    
#    display(p1)
    return p1
end

In [ ]:
function plotSynergyOldWay(thisPatient)
    gr()
    (conditionsForPat,namesForPatConds)=getPatConds(thisPatient)
    p1=plot()
    
    cA=palette(:seaborn_colorblind,length(conditionsForPat))


    # the first condition in the WT so lets store this
    WTCellNumbers=df[!,Symbol(conditionsForPat[1])]
    
    # here we'll store how many cells are added by each mutations starting with zero
    currentTotal=zeros(size(WTCellNumbers))
    
    conditionIndex=1
    # we need to loop through all conditions except the first (WT) and last (all muts)
    for condition in conditionsForPat[2:end-1]
        # subtract the WT simulation from this condition to figure out how many cells are added (or subtracted) by this mutation
        cellsAddedByMutation=df[!,Symbol(condition)].-WTCellNumbers
        
        # add this to the current total to get the effect of adding all mutations together
        currentTotal=currentTotal.+cellsAddedByMutation        
        conditionIndex+=1
    end
    # we need to plot the additive effect, which is the cells added by all mutations together on top of the WT
    additiveEffect=currentTotal
    plot!(additiveEffect,label="additive",linecolor=:red, linealpha=0.75,linewidth=6)

    #effect of all mutations over the WT
    allMutsEffect=df[!,thisPatient].-WTCellNumbers
    
    plot!(allMutsEffect,label="all_muts",linecolor=:black, linealpha=0.75,linewidth=6)
    thisLineColor=:black
    
    # the synergy region we want to shade is between the additive effect and all must sim    
    lower=minimum(hcat(allMutsEffect,additiveEffect),dims=2)
    upper=maximum(hcat(allMutsEffect,additiveEffect),dims=2)
    mutationEffect=df[!,thisPatient].-additiveEffect
    # plot that region with a red/blue ribbon
    plot!([allMutsEffect allMutsEffect],fillrange=[lower upper],fillalpha=0.35,c = [:red :blue],label = ["> additive (synergistic)" "<= additive"],linecolor=thisLineColor, linealpha=0.75,linewidth=3)
    plot!(xticks = (0:60*24:maxTimeTC, string.(0:24:Int64(maxTimeTC))))
    plot!(ylabel="cell number")
    plot!(xlabel="time (h)")
    plot!(legend=:right,dpi=300,margin=10Plots.mm)
#    display(p1)

    return p1
end

In [ ]:
function plotSynergy(thisPatient)
    gr()
    (conditionsForPat,namesForPatConds)=getPatConds(thisPatient)
    p1=plot()
    
    cA=palette(:seaborn_colorblind,length(conditionsForPat))


    # the first condition in the WT so lets store this
    WTCellNumbers=df[!,Symbol(conditionsForPat[1])]
    
    # here we'll store how many cells are added by each mutations starting with zero
    currentTotal=zeros(size(WTCellNumbers))
    
    conditionIndex=1
    # we need to loop through all conditions except the first (WT) and last (all muts)
    for condition in conditionsForPat[2:end-1]
        # subtract the WT simulation from this condition to figure out how many cells are added (or subtracted) by this mutation
        cellsAddedByMutation=df[!,Symbol(condition)].-WTCellNumbers
        
        # add this to the current total to get the effect of adding all mutations together
        currentTotal=currentTotal.+cellsAddedByMutation        
        conditionIndex+=1
    end
    # we need to plot the additive effect, which is the cells added by all mutations together on top of the WT
    additiveEffect=currentTotal.+WTCellNumbers
    plot!(additiveEffect,label="additive",linecolor=:red, linealpha=0.75,linewidth=6)

    # this is the all muts sim
    plot!(df[!,thisPatient],label="all_muts",linecolor=:black, linealpha=0.75,linewidth=6)
    thisLineColor=:black
    
    # the synergy region we want to shade is between the additive effect and all must sim    
    lower=minimum(hcat(df[!,thisPatient],additiveEffect),dims=2)
    upper=maximum(hcat(df[!,thisPatient],additiveEffect),dims=2)
    
    # plot that region with a red/blue ribbon
    plot!([df[!,thisPatient] df[!,thisPatient]],fillrange=[lower upper],fillalpha=0.35,c = [:red :blue],label = ["> additive (synergistic)" "<= additive"],linecolor=thisLineColor, linealpha=0.75,linewidth=3)
    plot!(xticks = (0:60*24:maxTimeTC, string.(0:24:Int64(maxTimeTC))))
    plot!(ylabel="cell number")
    plot!(xlabel="time (h)")
    plot!(legend=:right,dpi=300,margin=10Plots.mm)
#    display(p1)

    return p1
end

In [ ]:
function calculateMetrics(thisPatient)
    gr()
    (conditionsForPat,namesForPatConds)=getPatConds(thisPatient)
    
    # the first condition in the WT so lets store this
    WTCellNumbers=df[!,Symbol(conditionsForPat[1])]
    
    # here we'll store how many cells are added by each mutations starting with zero
    currentTotal=zeros(size(WTCellNumbers))
    
    conditionIndex=1
    # we need to loop through all conditions except the first (WT) and last (all muts)
    for condition in conditionsForPat[2:end-1]
        # subtract the WT simulation from this condition to figure out how many cells are added (or subtracted) by this mutation
        cellsAddedByMutation=df[!,Symbol(condition)].-WTCellNumbers
        
        # add this to the current total to get the effect of adding all mutations together
        currentTotal=currentTotal.+cellsAddedByMutation        
        conditionIndex+=1
    end
    # we need to plot the additive effect, which is the cells added by all mutations together on top of the WT
    additiveEffect=currentTotal.+WTCellNumbers


    # calculate and return a bunch of metrics
    peakCN=maximum(df[!,thisPatient]) # peak of allMuts simulation
    AUCCN=sum(df[!,thisPatient]) # AUC of allMuts simulation
    peakEIA=maximum(additiveEffect) # peak of allMuts simulation minus WT
    AUCEIA=sum(additiveEffect) # sum allMuts simulation minus WT
    peakSyn=maximum(df[!,thisPatient].-additiveEffect)
    AUCSyn=sum(df[!,thisPatient].-additiveEffect)
    onlyPosSyn=df[!,thisPatient].-additiveEffect
    onlyPosSyn[onlyPosSyn.<0].=0
    normalisedSynergy=AUCSyn/AUCEIA
    sumOnlyPosSyn=sum(onlyPosSyn)
    return (peakCN,AUCCN,peakEIA,AUCEIA,peakSyn,AUCSyn,sumOnlyPosSyn,normalisedSynergy)
end

In [ ]:
function plotAllPlots(patient)
    p1=plotCellNumbers(patient)
    p2=plotSynergyOldWay(patient)
    p3=plotSynergy(patient)
    display(plot(p1,p2,p3,layout = (1,3),size=(3000,600)))
    savefig("plots/"*patient*".png")
end

In [ ]:
#put plots in a folder
mkpath("plots")

#patientsToRun
patientsToRun=patients

#fill this array with all the metrics for all patients
allCalculatedMetrics=zeros(length(patientsToRun),8)
patientIndex=1

#loop through every patient
for patient in patientsToRun
    println(patient)
    
    #calculate all metrics for the patient and print them
    (peakCN,AUCCN,peakEIA,AUCEIA,peakSyn,AUCSyn,sumOnlyPosSyn,normalisedSyn)=calculateMetrics(patient)
    
    println("PeakCN: "*string(peakCN))
    println("AUCCN: "*string(AUCCN))
    println("peakEIA: "*string(peakEIA))
    println("AUCEIA: "*string(AUCEIA))
    println("peakSyn: "*string(peakSyn))
    println("AUCSyn: "*string(AUCSyn))
    println("Normalised Synergy: "*string(normalisedSyn))

    println("AUCOnlyPosSyn (AUC of red area only): "*string(sumOnlyPosSyn))    

    #save all the calulcated metrics to an array to write out afterwards
    allCalculatedMetrics[patientIndex,:]=[peakCN,AUCCN,peakEIA,AUCEIA,peakSyn,AUCSyn,sumOnlyPosSyn,normalisedSyn]

    #print the plots for the patient to check the metrics make sense.
    plotAllPlots(patient)
    patientIndex+=1
end
#names of the metrics for the data frame
metricNames=["peakCN","AUCCN","peakEIA","AUCEIA","peakSyn","AUCSyn","sumOnlyPosSyn","normalisedSyn"]
df=DataFrame(allCalculatedMetrics,Symbol.(metricNames))
df[!,:patientName] = patientsToRun
df=df[!, [end, 1,2,3,4,5,6,7,8]]
CSV.write("outputFileMetrics.csv",df)

In [24]:
Param_combs_output = DataFrame(CSV.File("paper_revisions/Param_combs_outputFile.csv"))

Row,combName,peakCN,AUCCN,peakEIA,AUCEIA,peakSyn,AUCSyn,sumOnlyPosSyn,normalisedSyn
,String15,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,"[1, 2]",0.0,0.0,99.0,95843.0,58.0,-95843.0,34826.0,-1.0
2,"[1, 3]",114.0,280651.0,152.0,300955.0,40.0,-20304.0,27284.0,-0.0674652
3,"[1, 4]",288.0,548497.0,404.0,697769.0,39.0,-149272.0,20006.0,-0.213928
4,"[1, 5]",99.0,203313.0,99.0,216315.0,32.0,-13002.0,63.0,-0.0601068
5,"[1, 6]",99.0,207858.0,99.0,230604.0,28.0,-22746.0,72.0,-0.0986366
6,"[1, 7]",99.0,224322.0,99.0,265028.0,1.0,-40706.0,1.0,-0.153591
7,"[1, 8]",0.0,0.0,99.0,193948.0,0.0,-193948.0,0.0,-1.0
8,"[1, 9]",99.0,193007.0,99.0,192694.0,32.0,313.0,511.0,0.00162434
9,"[1, 10]",99.0,184362.0,99.0,173480.0,33.0,10882.0,14254.0,0.0627277


In [27]:
#### Mut/Param data
muts2Params = DataFrame(CSV.File("muts2ParamsNEW.csv"))
# concatenate gene and alteration
muts2Params[!, "GeneAlt"] = string.(muts2Params[!, "GENE"], ":", muts2Params[!, "ALTERATION"])
## remove zeros
muts2Params = muts2Params[(muts2Params.PARAMETER .!= "0"), :]

Row,TYPE,ALTERATION,GENE,PARAMETER,AMOUNT,GeneAlt
,String15,String,String7,String31,Float64,String
1,CNA,AMP,BAD,k1_81-apoptosis,1.5,BAD:AMP
2,CNA,AMP,BAX,k1_86-apoptosis,1.5,BAX:AMP
3,CNA,AMP,BCL2,k1_bcl2Exp-linking,1.5,BCL2:AMP
4,CNA,HETLOSS,CASP3,k1_101-apoptosis,0.5,CASP3:HETLOSS
5,CNA,AMP,CCNB1,k2_1a-cellCycle,1.5,CCNB1:AMP
6,CNA,AMP,CCNE1,k3_7-cellCycle,1.5,CCNE1:AMP
7,CNA,HETLOSS,CDKN1B,k2_5-cellCycle,0.5,CDKN1B:HETLOSS
8,CNA,AMP,CYCS,k1_113-apoptosis,1.5,CYCS:AMP
9,CNA,AMP,MCL1,k1_bcl2Exp-linking,1.5,MCL1:AMP


In [28]:
param_val = muts2Params[!, 4:5]
param_val_unique = unique!(param_val)

Row,PARAMETER,AMOUNT
,String31,Float64
1,k1_81-apoptosis,1.5
2,k1_86-apoptosis,1.5
3,k1_bcl2Exp-linking,1.5
4,k1_101-apoptosis,0.5
5,k2_1a-cellCycle,1.5
6,k3_7-cellCycle,1.5
7,k2_5-cellCycle,0.5
8,k1_113-apoptosis,1.5
9,k2_scalingNumber-NFkB,1.5


In [29]:
delete!(param_val, [12])

Row,PARAMETER,AMOUNT
,String31,Float64
1,k1_81-apoptosis,1.5
2,k1_86-apoptosis,1.5
3,k1_bcl2Exp-linking,1.5
4,k1_101-apoptosis,0.5
5,k2_1a-cellCycle,1.5
6,k3_7-cellCycle,1.5
7,k2_5-cellCycle,0.5
8,k1_113-apoptosis,1.5
9,k2_scalingNumber-NFkB,1.5


In [31]:
using Combinatorics

In [32]:
param_combs = collect(combinations(1:22,2))

231-element Vector{Vector{Int64}}:
 [1, 2]
 [1, 3]
 [1, 4]
 [1, 5]
 [1, 6]
 [1, 7]
 [1, 8]
 [1, 9]
 [1, 10]
 [1, 11]
 [1, 12]
 [1, 13]
 [1, 14]
 ⋮
 [17, 21]
 [17, 22]
 [18, 19]
 [18, 20]
 [18, 21]
 [18, 22]
 [19, 20]
 [19, 21]
 [19, 22]
 [20, 21]
 [20, 22]
 [21, 22]

In [34]:
params = param_val.PARAMETER

22-element PooledArrays.PooledVector{String31, UInt32, Vector{UInt32}}:
 "k1_81-apoptosis"
 "k1_86-apoptosis"
 "k1_bcl2Exp-linking"
 "k1_101-apoptosis"
 "k2_1a-cellCycle"
 "k3_7-cellCycle"
 "k2_5-cellCycle"
 "k1_113-apoptosis"
 "k2_scalingNumber-NFkB"
 "k1_tIkBb-NFkB"
 "kmax_IkBeSynth-NFkB"
 "k1_108-apoptosis"
 "k1_s_Blimp1-differentiation"
 "k1_26-cellCycle"
 "basal_cRelSynth-linking"
 "k1_s_Bcl6-differentiation"
 "k1_mycExp-linking"
 "k1_3a-cellCycle"
 "k1_s_IRF4-linking"
 "kmax_IkBaSynth-NFkB"
 "k1_e_pax5-differentiation"
 "k1_mycTsl-linking"

In [43]:
Param_combs_output.combName = parse.(String31 ,Param_combs_output.combName)

LoadError: MethodError: no method matching parse(::Type{String31}, ::String15)
[0mClosest candidates are:
[0m  parse([91m::Type{P}[39m, ::AbstractString; kwargs...) where P<:FilePathsBase.AbstractPath at /home/richard/.julia/packages/FilePathsBase/9kSEl/src/path.jl:77
[0m  parse([91m::Type{T}[39m, ::AbstractString) where T<:Complex at parse.jl:381
[0m  parse([91m::Type{T}[39m, ::AbstractString; base) where T<:Integer at parse.jl:240
[0m  ...

In [41]:
Param_combs_output

Row,combName,peakCN,AUCCN,peakEIA,AUCEIA,peakSyn,AUCSyn,sumOnlyPosSyn,normalisedSyn
,String15,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,"[1, 2]",0.0,0.0,99.0,95843.0,58.0,-95843.0,34826.0,-1.0
2,"[1, 3]",114.0,280651.0,152.0,300955.0,40.0,-20304.0,27284.0,-0.0674652
3,"[1, 4]",288.0,548497.0,404.0,697769.0,39.0,-149272.0,20006.0,-0.213928
4,"[1, 5]",99.0,203313.0,99.0,216315.0,32.0,-13002.0,63.0,-0.0601068
5,"[1, 6]",99.0,207858.0,99.0,230604.0,28.0,-22746.0,72.0,-0.0986366
6,"[1, 7]",99.0,224322.0,99.0,265028.0,1.0,-40706.0,1.0,-0.153591
7,"[1, 8]",0.0,0.0,99.0,193948.0,0.0,-193948.0,0.0,-1.0
8,"[1, 9]",99.0,193007.0,99.0,192694.0,32.0,313.0,511.0,0.00162434
9,"[1, 10]",99.0,184362.0,99.0,173480.0,33.0,10882.0,14254.0,0.0627277


In [47]:
insertcols!(Param_combs_output ,10, :combName2 => "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")

Row,combName,peakCN,AUCCN,peakEIA,AUCEIA,peakSyn,AUCSyn,sumOnlyPosSyn,normalisedSyn,combName2
,String15,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String
1,"[1, 2]",0.0,0.0,99.0,95843.0,58.0,-95843.0,34826.0,-1.0,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
2,"[1, 3]",114.0,280651.0,152.0,300955.0,40.0,-20304.0,27284.0,-0.0674652,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
3,"[1, 4]",288.0,548497.0,404.0,697769.0,39.0,-149272.0,20006.0,-0.213928,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
4,"[1, 5]",99.0,203313.0,99.0,216315.0,32.0,-13002.0,63.0,-0.0601068,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
5,"[1, 6]",99.0,207858.0,99.0,230604.0,28.0,-22746.0,72.0,-0.0986366,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
6,"[1, 7]",99.0,224322.0,99.0,265028.0,1.0,-40706.0,1.0,-0.153591,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
7,"[1, 8]",0.0,0.0,99.0,193948.0,0.0,-193948.0,0.0,-1.0,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
8,"[1, 9]",99.0,193007.0,99.0,192694.0,32.0,313.0,511.0,0.00162434,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
9,"[1, 10]",99.0,184362.0,99.0,173480.0,33.0,10882.0,14254.0,0.0627277,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx


In [48]:
for i in 1:231
    comb = param_combs[i]
    p1 = params[comb[1]]
    p2 = params[comb[2]]
    param_comb = string(p1, ":", p2)
    Param_combs_output[i,10] = param_comb
end

In [49]:
Param_combs_output

Row,combName,peakCN,AUCCN,peakEIA,AUCEIA,peakSyn,AUCSyn,sumOnlyPosSyn,normalisedSyn,combName2
,String15,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String
1,"[1, 2]",0.0,0.0,99.0,95843.0,58.0,-95843.0,34826.0,-1.0,k1_81-apoptosis:k1_86-apoptosis
2,"[1, 3]",114.0,280651.0,152.0,300955.0,40.0,-20304.0,27284.0,-0.0674652,k1_81-apoptosis:k1_bcl2Exp-linking
3,"[1, 4]",288.0,548497.0,404.0,697769.0,39.0,-149272.0,20006.0,-0.213928,k1_81-apoptosis:k1_101-apoptosis
4,"[1, 5]",99.0,203313.0,99.0,216315.0,32.0,-13002.0,63.0,-0.0601068,k1_81-apoptosis:k2_1a-cellCycle
5,"[1, 6]",99.0,207858.0,99.0,230604.0,28.0,-22746.0,72.0,-0.0986366,k1_81-apoptosis:k3_7-cellCycle
6,"[1, 7]",99.0,224322.0,99.0,265028.0,1.0,-40706.0,1.0,-0.153591,k1_81-apoptosis:k2_5-cellCycle
7,"[1, 8]",0.0,0.0,99.0,193948.0,0.0,-193948.0,0.0,-1.0,k1_81-apoptosis:k1_113-apoptosis
8,"[1, 9]",99.0,193007.0,99.0,192694.0,32.0,313.0,511.0,0.00162434,k1_81-apoptosis:k2_scalingNumber-NFkB
9,"[1, 10]",99.0,184362.0,99.0,173480.0,33.0,10882.0,14254.0,0.0627277,k1_81-apoptosis:k1_tIkBb-NFkB


In [51]:
Param_combs_output = Param_combs_output[(Param_combs_output.peakSyn .!= 0), :]

Row,combName,peakCN,AUCCN,peakEIA,AUCEIA,peakSyn,AUCSyn,sumOnlyPosSyn,normalisedSyn,combName2
,String15,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String
1,"[1, 7]",99.0,224322.0,99.0,265028.0,1.0,-40706.0,1.0,-0.153591,k1_81-apoptosis:k2_5-cellCycle
2,"[1, 14]",99.0,203541.0,99.0,220468.0,6.0,-16927.0,19.0,-0.0767776,k1_81-apoptosis:k1_26-cellCycle
3,"[2, 13]",99.0,205911.0,99.0,222434.0,6.0,-16523.0,37.0,-0.0742827,k1_86-apoptosis:k1_s_Blimp1-differentiation
4,"[1, 15]",99.0,215058.0,99.0,244297.0,8.0,-29239.0,11.0,-0.119686,k1_81-apoptosis:basal_cRelSynth-linking
5,"[1, 16]",99.0,217309.0,99.0,241728.0,8.0,-24419.0,2089.0,-0.101019,k1_81-apoptosis:k1_s_Bcl6-differentiation
6,"[2, 14]",99.0,217479.0,99.0,246263.0,8.0,-28784.0,12.0,-0.116883,k1_86-apoptosis:k1_26-cellCycle
7,"[7, 16]",111.0,295031.0,125.0,314276.0,9.0,-19245.0,268.0,-0.061236,k2_5-cellCycle:k1_s_Bcl6-differentiation
8,"[11, 14]",134.0,309557.0,138.0,321331.0,10.0,-11774.0,879.0,-0.0366413,kmax_IkBeSynth-NFkB:k1_26-cellCycle
9,"[2, 15]",99.0,219845.0,99.0,243694.0,12.0,-23849.0,2128.0,-0.0978645,k1_86-apoptosis:basal_cRelSynth-linking


In [52]:
sort!(Param_combs_output, [:peakSyn], rev=true)

Row,combName,peakCN,AUCCN,peakEIA,AUCEIA,peakSyn,AUCSyn,sumOnlyPosSyn,normalisedSyn,combName2
,String15,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String
1,"[4, 15]",1117.0,1.83108e6,706.0,1.12493e6,812.0,706148.0,707467.0,0.627726,k1_101-apoptosis:basal_cRelSynth-linking
2,"[3, 22]",744.0,1.28692e6,612.0,965765.0,358.0,321152.0,325085.0,0.332536,k1_bcl2Exp-linking:k1_mycTsl-linking
3,"[2, 19]",558.0,1.06318e6,536.0,904847.0,280.0,158335.0,183959.0,0.174985,k1_86-apoptosis:k1_s_IRF4-linking
4,"[3, 15]",480.0,801778.0,456.0,728117.0,278.0,73661.0,94320.0,0.101166,k1_bcl2Exp-linking:basal_cRelSynth-linking
5,"[3, 20]",661.0,1.17436e6,440.0,868920.0,265.0,305441.0,305441.0,0.351518,k1_bcl2Exp-linking:kmax_IkBaSynth-NFkB
6,"[17, 22]",577.0,840815.0,379.0,668890.0,237.0,171925.0,172963.0,0.25703,k1_mycExp-linking:k1_mycTsl-linking
7,"[4, 14]",488.0,951672.0,459.0,845620.0,232.0,106052.0,111972.0,0.125413,k1_101-apoptosis:k1_26-cellCycle
8,"[9, 15]",505.0,829126.0,516.0,789035.0,226.0,40091.0,71568.0,0.0508102,k2_scalingNumber-NFkB:basal_cRelSynth-linking
9,"[4, 13]",612.0,1.00853e6,439.0,848189.0,218.0,160339.0,160339.0,0.189037,k1_101-apoptosis:k1_s_Blimp1-differentiation


In [57]:
sort!(Param_combs_output, [:combName])

Row,combName,peakCN,AUCCN,peakEIA,AUCEIA,peakSyn,AUCSyn,sumOnlyPosSyn,normalisedSyn,combName2
,String15,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String
1,"[1, 10]",99.0,184362.0,99.0,173480.0,33.0,10882.0,14254.0,0.0627277,k1_81-apoptosis:k1_tIkBb-NFkB
2,"[1, 12]",99.0,210397.0,99.0,232440.0,13.0,-22043.0,15.0,-0.0948331,k1_81-apoptosis:k1_108-apoptosis
3,"[1, 13]",99.0,203820.0,99.0,220134.0,16.0,-16314.0,2182.0,-0.0741094,k1_81-apoptosis:k1_s_Blimp1-differentiation
4,"[1, 14]",99.0,203541.0,99.0,220468.0,6.0,-16927.0,19.0,-0.0767776,k1_81-apoptosis:k1_26-cellCycle
5,"[1, 15]",99.0,215058.0,99.0,244297.0,8.0,-29239.0,11.0,-0.119686,k1_81-apoptosis:basal_cRelSynth-linking
6,"[1, 16]",99.0,217309.0,99.0,241728.0,8.0,-24419.0,2089.0,-0.101019,k1_81-apoptosis:k1_s_Bcl6-differentiation
7,"[1, 17]",236.0,426162.0,336.0,521039.0,22.0,-94877.0,12426.0,-0.182092,k1_81-apoptosis:k1_mycExp-linking
8,"[1, 18]",99.0,166667.0,99.0,143125.0,38.0,23542.0,25134.0,0.164486,k1_81-apoptosis:k1_3a-cellCycle
9,"[1, 19]",99.0,203541.0,99.0,220436.0,32.0,-16895.0,51.0,-0.0766436,k1_81-apoptosis:k1_s_IRF4-linking


In [58]:
show(Param_combs_output, allrows=true)

218×10 DataFrame
 Row │ combName  peakCN   AUCCN           peakEIA  AUCEIA          peakSyn  AU ⋯
     │ String15  Float64  Float64         Float64  Float64         Float64  Fl ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │ [1, 10]      99.0  184362.0           99.0  173480.0           33.0     ⋯
   2 │ [1, 12]      99.0  210397.0           99.0  232440.0           13.0   -
   3 │ [1, 13]      99.0  203820.0           99.0  220134.0           16.0   -
   4 │ [1, 14]      99.0  203541.0           99.0  220468.0            6.0   -
   5 │ [1, 15]      99.0  215058.0           99.0  244297.0            8.0   - ⋯
   6 │ [1, 16]      99.0  217309.0           99.0  241728.0            8.0   -
   7 │ [1, 17]     236.0  426162.0          336.0  521039.0           22.0   -
   8 │ [1, 18]      99.0  166667.0           99.0  143125.0           38.0
   9 │ [1, 19]      99.0  203541.0           99.0  220436.0           32.0   - ⋯
  10 │ [1, 20]     114.0  2

In [53]:
CSV.write("Param_combs_output.csv", Param_combs_output)

"Param_combs_output.csv"